In [100]:
from haystack.document_stores.memory import InMemoryDocumentStore
from haystack.nodes.retriever.dense import DensePassageRetriever
from haystack.nodes.reader import FARMReader
from haystack.pipelines import ExtractiveQAPipeline, GenerativeQAPipeline
from haystack.utils import print_answers
from haystack.nodes import RAGenerator
from haystack.nodes import Shaper, PromptNode, PromptTemplate, PromptModel, EmbeddingRetriever
from haystack import Pipeline

from bs4 import BeautifulSoup
import requests, json, lxml

headers = {
    'User-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

query = input("Enter your search query: ")

# https://requests.readthedocs.io/en/latest/user/quickstart/#passing-parameters-in-urls
params = {
  'q': query,  # query 
  'gl': 'us',    # country to search from
  'hl': 'en'     # language
}

# https://requests.readthedocs.io/en/latest/user/quickstart/#timeouts
html = requests.get("https://www.google.com/search?q="+params['q'], headers=headers, params=params, timeout=30)
soup = BeautifulSoup(html.text, 'lxml')


data = []

for result in soup.select('.tF2Cxc'):
    title = result.select_one('.DKV0Md').text
    link = result.select_one('.yuRUbf a')['href']
    try:
        snippet = result.select_one('#rso .lyLwlc').text
        data.append({
           'title': title,
           'link': link,
           'snippet': snippet
        })
    except:
        pass



#print(document_store.get_all_documents())
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

    
document_store = InMemoryDocumentStore()
document_store.delete_documents()
# Add search snippets to document store
for result in data:
    document_store.write_documents([{
        "content": result["snippet"],
        "meta": {
            "name": result["title"],
            "url": result["link"]
        }
    }])
    
# Add documents embeddings to index

document_store.update_embeddings(retriever=retriever)    


#print(json.dumps(data, indent=2, ensure_ascii=False))

# Delete existing documents in documents store

# Initialize document store


# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=5,
    max_length=200,
    min_length=20,
    embed_title=True,
    num_beams=5,
    retriever=retriever
)
pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
res = pipe.run(query=query, params={"Generator": {"top_k": 5}, "Retriever": {"top_k": 5}})
print_answers(res, details="minimum")


Enter your search query:  How has Neoplatonism influenced today's ideas?


Updating Embedding:   0%|          | 0/10 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/16 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/16 [00:00<?, ? Docs/s]

"Query: How has Neoplatonism influenced today's ideas?"
'Answers:'
[   {   'answer': ' its synthesis of ancient greek philosophy and the '
                  'abrahamic traditions andotsotsi'},
    {   'answer': ' its synthesis of ancient greek philosophy and the '
                  'abrahamic traditions andotsotsis'},
    {   'answer': ' its synthesis of ancient greek philosophy and the '
                  'abrahamic traditions andotsb.'},
    {   'answer': ' its synthesis of ancient greek philosophy and the '
                  'abrahamic traditions andotss'},
    {   'answer': ' its synthesis of ancient greek philosophy and the '
                  'abrahamic traditions andotss.'}]


In [106]:
data

[{'title': 'Neoplatonism - Stanford Encyclopedia of Philosophy',
  'link': 'https://plato.stanford.edu/entries/neoplatonism/',
  'snippet': 'by C Wildberg · 2016 · Cited by 57 — It may even be true to say that even more than the writings of Plato and Aristotle themselves Neoplatonic ideas have continued to influence\xa0...'},
 {'title': 'Christian Platonism, Neoplatonism and Modern Naturalism',
  'link': 'https://credomag.com/2022/01/christian-platonism-neoplatonism-and-modern-naturalism/',
  'snippet': 'Jan 12, 2022 — Neoplatonism was the most potent and influential form of Platonism at that time, and it was both a philosophy and a mystical religion. But it\xa0...'},
 {'title': 'Neoplatonism',
  'link': 'https://www.infoplease.com/encyclopedia/religion/philosophy/concepts/neoplatonism/the-impact-of-neoplatonism',
  'snippet': "The system influenced medieval Jewish and Arab philosophy, and G. W. F. Hegel's metaphysics had Neoplatonic ingredients. Neoplatonic metaphysics and aesthetics\

{'query': "How has Neoplatonism influenced today's ideas?",
 'answers': [<Answer {'answer': ' its synthesis of ancient greek philosophy and the abrahamic traditions andotsotsi', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['4329f558e0539f550094a2ab0b041dd6', 'dddc81fd36339cca63fbb6b0badd7237', '31f1c1a636b528c6b16af98371cb19cf', '55ac2ed0b844950770e21d7103a991f8', '78b591bd14f8dad66f28fd4f049be2ba'], 'meta': {'doc_scores': [0.6971465544966285, 0.6900900149730781, 0.685922565979213, 0.6824336064790444, 0.68232519791714], 'content': ['Feb 9, 2023 — The greatest influence of Neoplatonism is its synthesis of ancient Greek philosophy and the Abrahamic traditions, including Christianity, Islam,\xa0...', "The system influenced medieval Jewish and Arab philosophy, and G. W. F. Hegel's metaphysics had Neoplatonic ingredients. Neoplatonic metaphysics and aesthetics\xa0...", 'Neoplatonism had an enduring influence 